In [ ]:
import matplotlib.pyplot as plt

from pymedphys.dcm import coords_and_dose_from_dcm

from IPython.display import clear_output

In [ ]:
import numpy as np

In [ ]:
distance_mm_threshold = 3
dose_percent_threshold = 3

lower_percent_dose_cutoff = 20

normalisation = None

In [ ]:
dcm_ref_filepath = "../../../../Documents/RD.Ecl_WOmask.dcm"
dcm_eval_filepath = "../../../../Documents/RD.Pin.dcm"

coords_reference, dose_reference = coords_and_dose_from_dcm(
    dcm_ref_filepath)
coords_evaluation, dose_evaluation = coords_and_dose_from_dcm(
    dcm_eval_filepath)

coords_evaluation = [
    coord[1:-1]
    for coord in coords_evaluation
]

dose_evaluation = dose_evaluation[1:-1,1:-1,1:-1]

# coords_reference = list(coords_reference)
# coords_reference[0] = coords_reference[0] + 20

In [ ]:
np.shape(dose_reference)

In [ ]:
len(coords_reference[0])

In [ ]:
len(coords_reference[1])

In [ ]:
len(coords_reference[2])

In [ ]:
xx_ref, yy_ref, zz_ref = np.meshgrid(*coords_reference, indexing='ij')

In [ ]:
np.mean(dose_reference)

In [ ]:
if normalisation is None:
    normalisation = np.max(dose_reference)

dose_threshold = dose_percent_threshold / 100 * normalisation
lower_dose_cutoff = lower_percent_dose_cutoff / 100 * normalisation

In [ ]:
for _ in range(10):
    lower_dose_cutoff = lower_percent_dose_cutoff / 100 * normalisation
    
    sorted_ref_dose = np.sort(np.ravel(dose_reference))
    sorted_ref_dose = sorted_ref_dose[sorted_ref_dose > lower_dose_cutoff]
    ref_98 = np.floor(len(sorted_ref_dose) * 0.98).astype(int)
    normalisation = sorted_ref_dose[ref_98]
    

normalisation

In [ ]:
dose_threshold = dose_percent_threshold / 100 * normalisation
lower_dose_cutoff = lower_percent_dose_cutoff / 100 * normalisation

In [ ]:
gamma_array = np.ones_like(dose_evaluation).astype(np.float) * np.nan

In [ ]:
len(coords_evaluation[0])

In [ ]:
len(coords_evaluation[1])

In [ ]:
mesh_index = np.meshgrid(*[
    np.arange(len(coord_eval))
    for coord_eval in coords_evaluation
])

eval_index = np.reshape(np.array(mesh_index), (3,-1))
run_index = np.arange(np.shape(eval_index)[1])
np.random.shuffle(run_index)
run_index

In [ ]:
i, j, k = eval_index[:, 0]
i

In [ ]:
run_index = np.arange(np.shape(eval_index)[1])
np.random.shuffle(run_index)
run_index

gamma_array = np.ones_like(dose_evaluation).astype(np.float) * np.nan

for counter, point_index in enumerate(run_index):
    i, j, k = eval_index[:, point_index]
    eval_x = coords_evaluation[0][i]
    eval_y = coords_evaluation[1][j]
    eval_z = coords_evaluation[2][k]
    
    if dose_evaluation[i, j, k] < lower_dose_cutoff:
        continue
    
    distance = np.sqrt(
        (xx_ref - eval_x)**2 +
        (yy_ref - eval_y)**2 +
        (zz_ref - eval_z)**2
    )
    
    dose_diff = dose_evaluation[i, j, k] - dose_reference

    gamma = np.min(np.sqrt((dose_diff/dose_threshold)**2 + (distance/distance_mm_threshold)**2))

    gamma_array[i,j,k] = gamma
    
    valid_gamma = gamma_array[np.invert(np.isnan(gamma_array))]
    percent_pass = 100 * np.sum(valid_gamma < 1) / len(valid_gamma)
    
    if counter // 30 == counter / 30:
        clear_output()
    print("complete percent, i, j, k, gamma, pass_rate = {}".format((
        counter / np.shape(eval_index)[1] * 100,
        i, j, k, 
        gamma, percent_pass)))

In [ ]:
# for i, eval_x in enumerate(coords_evaluation[0]):
#     print('i = {}'.format(i))
#     for j, eval_y in enumerate(coords_evaluation[1]):
#         print('j = {}'.format(j))
#         for k, eval_z in enumerate(coords_evaluation[2]):
#             distance = np.sqrt(
#                 (xx_ref - eval_x)**2 +
#                 (yy_ref - eval_y)**2 +
#                 (zz_ref - eval_z)**2
#             )
            
#             dose_diff = dose_evaluation[i, j, k] - dose_reference
            
#             gamma = np.min(np.sqrt((dose_diff/dose_threshold)**2 + (distance/distance_mm_threshold)**2))
            
#             gamma_array[i,j,k] = gamma
            
#         gamma_just_calced = gamma_array[i,j,:]
#         print("min: {}, max: {}, mean: {}".format(
#             np.min(gamma_just_calced),
#             np.max(gamma_just_calced),
#             np.mean(gamma_just_calced)))
            

In [ ]:
gamma_above_threshold = gamma_array[dose_evaluation>lower_dose_cutoff]

In [ ]:
100*np.sum(gamma_above_threshold < 1) / len(gamma_above_threshold)